In [572]:

import torch 

Example sentence: `I love you`

In [573]:
input = torch.tensor([0,1,2])
input,input.shape

(tensor([0, 1, 2]), torch.Size([3]))

In [574]:
from torch.nn import Embedding

embedding = Embedding(3,4)
embedded_sentence =embedding(input)
embedded_sentence,embedded_sentence.shape


(tensor([[ 1.2990,  0.6139,  0.3475,  1.5683],
         [-0.5906, -0.2768, -1.7423,  0.7687],
         [-1.3571,  0.3477,  0.4122,  0.2421]], grad_fn=<EmbeddingBackward0>),
 torch.Size([3, 4]))

In [575]:

q_vector=[]
k_vector=[]
v_vector=[]
q_vector.clear()
k_vector.clear()
v_vector.clear()

Wq = torch.rand(4,3)
Wk = torch.rand(4,3)
Wv = torch.rand(4,3)


for word in embedded_sentence:
    q_vector.append(torch.matmul(word.unsqueeze(0),Wq))
    k_vector.append(torch.matmul(word.unsqueeze(0),Wk))
    v_vector.append(torch.matmul(word.unsqueeze(0),Wv))


In [576]:
q_vector

[tensor([[1.7892, 1.4993, 1.0465]], grad_fn=<MmBackward0>),
 tensor([[-0.5220, -1.1909, -0.5180]], grad_fn=<MmBackward0>),
 tensor([[ 0.3214, -0.5950, -0.4371]], grad_fn=<MmBackward0>)]

In [577]:
k_vector , k_vector[0].shape

([tensor([[1.4714, 1.5221, 1.3076]], grad_fn=<MmBackward0>),
  tensor([[-0.8596, -0.9847, -1.0301]], grad_fn=<MmBackward0>),
  tensor([[-1.1436, -0.5418,  0.2760]], grad_fn=<MmBackward0>)],
 torch.Size([1, 3]))

In [578]:
v_vector

[tensor([[1.7194, 2.1748, 1.6212]], grad_fn=<MmBackward0>),
 tensor([[-0.1163, -1.1907, -1.3755]], grad_fn=<MmBackward0>),
 tensor([[ 0.1521,  0.1838, -0.4775]], grad_fn=<MmBackward0>)]

In [579]:
k_vector

[tensor([[1.4714, 1.5221, 1.3076]], grad_fn=<MmBackward0>),
 tensor([[-0.8596, -0.9847, -1.0301]], grad_fn=<MmBackward0>),
 tensor([[-1.1436, -0.5418,  0.2760]], grad_fn=<MmBackward0>)]

In [580]:
q_vector[0].shape

torch.Size([1, 3])

`Attention --> (I) x(I)  in I love you`

In [581]:
torch.dot(q_vector[0].squeeze(),k_vector[0].squeeze()).item()

6.283153533935547

In [582]:
q_vector[0]

tensor([[1.7892, 1.4993, 1.0465]], grad_fn=<MmBackward0>)

In [583]:
attention_scores=[]
attention_scores.clear()
def calculate_attention(q_vector,k_vector):
    for q in q_vector:
        word_score=[]
        for k in k_vector:
            score = torch.dot(q.squeeze(),k.squeeze()).item()
            word_score.append(score)

        attention_scores.append(word_score.copy())
       
        word_score.clear()
calculate_attention(q_vector,k_vector)


In [584]:
attention_scores

[[6.283153533935547, -4.092398643493652, -2.569571018218994],
 [-3.2582621574401855, 2.1550886631011963, 1.0992695093154907],
 [-1.0043386220932007, 0.75986647605896, -0.16579794883728027]]

In [585]:
k_vector[0].squeeze().shape

torch.Size([3])

`Normalized Attention Scores`

In [586]:
attention_scores = torch.tensor(attention_scores ) / torch.sqrt(torch.tensor(3))
attention_scores

tensor([[ 3.6276, -2.3627, -1.4835],
        [-1.8812,  1.2442,  0.6347],
        [-0.5799,  0.4387, -0.0957]])

In [587]:
activation_scores = torch.softmax(attention_scores, dim=1)
activation_scores

tensor([[0.9915, 0.0025, 0.0060],
        [0.0277, 0.6299, 0.3424],
        [0.1855, 0.5136, 0.3010]])

In [588]:
torch.sum(activation_scores[0])

tensor(1.0000)

In [589]:
v_vector

[tensor([[1.7194, 2.1748, 1.6212]], grad_fn=<MmBackward0>),
 tensor([[-0.1163, -1.1907, -1.3755]], grad_fn=<MmBackward0>),
 tensor([[ 0.1521,  0.1838, -0.4775]], grad_fn=<MmBackward0>)]

In [590]:
activation_scores[0].shape , v_vector[0].shape

(torch.Size([3]), torch.Size([1, 3]))

`S1 x v1`

In [591]:
torch.dot(activation_scores[0],v_vector[0].squeeze())

tensor(1.7199, grad_fn=<DotBackward0>)

In [592]:
activation_scores[0]

tensor([0.9915, 0.0025, 0.0060])

In [593]:
activation_scores[0].shape , v_vector[0].shape

(torch.Size([3]), torch.Size([1, 3]))

In [594]:
torch.zeros_like(v_vector[0].squeeze())

tensor([0., 0., 0.])

In [595]:
v_vector

[tensor([[1.7194, 2.1748, 1.6212]], grad_fn=<MmBackward0>),
 tensor([[-0.1163, -1.1907, -1.3755]], grad_fn=<MmBackward0>),
 tensor([[ 0.1521,  0.1838, -0.4775]], grad_fn=<MmBackward0>)]

In [600]:
activation_scores[0] * v_vector[0].squeeze()

tensor([1.7049, 0.0054, 0.0097], grad_fn=<MulBackward0>)

`New representation for each word`

In [601]:
SxV = []
for s in attention_scores:
    weighted_sum=torch.zeros(3)
    for i , v in enumerate(v_vector):
        weighted_sum += s[i] * v.squeeze()

    SxV.append(weighted_sum)


In [604]:
SxV

[tensor([ 6.2863, 10.4300,  9.8394], grad_fn=<AddBackward0>),
 tensor([-3.2826, -5.4561, -5.0643], grad_fn=<AddBackward0>),
 tensor([-1.0626, -1.8010, -1.4978], grad_fn=<AddBackward0>)]